In [1]:
import os
import numpy as np
#import pandas as pd
import matplotlib.pyplot as plt
from random import shuffle
#import pickle, datetime
import math

import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential, load_model
from keras.layers import Input, Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, Convolution2D, MaxPooling2D
from keras.layers.convolutional import ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras import optimizers
from keras.preprocessing import sequence
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.callbacks import ModelCheckpoint

import  PIL.Image

C:\Users\jdu12\Anaconda3\envs\tf\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
root_path = '/Users/jdu12/Desktop/research/'
model_dir = root_path + 'saved_models/AlexNet/'
print(model_dir)

/Users/jdu12/Desktop/research/saved_models/AlexNet/


## Data Loading

In [3]:
N_CATEGORY = 36
BATCH_SIZE = 128
train_data_dir = '/Users/jdu12/Desktop/research/input'

train_datagen = ImageDataGenerator(
    #rescale=1. / 255,
    horizontal_flip=False)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(227, 227),
    batch_size=BATCH_SIZE,
    class_mode='categorical')
#print(x_train.shape, y_train.shape, x_test.shape, y_test.shape, N_CATEGORY)

Found 5075 images belonging to 36 classes.


## AlexNet Architecture 

### Model Parameters 

In [4]:
DROPOUT = 0.5
model_input = Input(shape = (227, 227, 3))

# First convolutional Layer (96x11x11)
z = Conv2D(filters = 96, kernel_size = (11,11), strides = (4,4), activation = "relu")(model_input)
z = MaxPooling2D(pool_size = (3,3), strides=(2,2))(z)
z = BatchNormalization()(z)

# Second convolutional Layer (256x5x5)
z = ZeroPadding2D(padding = (2,2))(z)
z = Convolution2D(filters = 256, kernel_size = (5,5), strides = (1,1), activation = "relu")(z)
z = MaxPooling2D(pool_size = (3,3), strides=(2,2))(z)
z = BatchNormalization()(z)

# Rest 3 convolutional layers
z = ZeroPadding2D(padding = (1,1))(z)
z = Convolution2D(filters = 384, kernel_size = (3,3), strides = (1,1), activation = "relu")(z)

z = ZeroPadding2D(padding = (1,1))(z)
z = Convolution2D(filters = 384, kernel_size = (3,3), strides = (1,1), activation = "relu")(z)

z = ZeroPadding2D(padding = (1,1))(z)
z = Convolution2D(filters = 256, kernel_size = (3,3), strides = (1,1), activation = "relu")(z)

z = MaxPooling2D(pool_size = (3,3), strides=(2,2))(z)
z = Flatten()(z)

z = Dense(4096, activation="relu")(z)
z = Dropout(DROPOUT)(z)

z = Dense(4096, activation="relu")(z)
z = Dropout(DROPOUT)(z)

final_dim = 1 if N_CATEGORY == 2 else N_CATEGORY
final_act = "sigmoid" if N_CATEGORY == 2 else "softmax"
model_output = Dense(final_dim, activation=final_act)(z)

model = Model(model_input, model_output)
#model.summary()

In [5]:
'''
import time
import pylab as pl
from IPython import display
pl.style.use('ggplot')
%matplotlib inline

class Histories(keras.callbacks.Callback):

    def on_train_begin(self, logs={}):
        self.acc = []
        self.loss = []
        self.val_loss = []
        self.val_acc = []

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        self.acc.append(logs['acc'])
        self.loss.append(logs['loss'])
        self.val_acc.append(logs['val_acc'])
        self.val_loss.append(logs['val_loss'])
        
        pl.hold(True)
        plt.rcParams["figure.figsize"] = (8,5)
        pl.plot(self.acc, 'r')
        pl.plot(self.loss, 'b')
        pl.plot(self.val_acc, 'g')
        pl.plot(self.val_loss, 'k')
        pl.legend(['Train acc','Train loss','Valid acc', 'Valid loss'], loc=2)
        display.clear_output(wait=True)
        display.display(pl.gcf())
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return
'''

'\nimport time\nimport pylab as pl\nfrom IPython import display\npl.style.use(\'ggplot\')\n%matplotlib inline\n\nclass Histories(keras.callbacks.Callback):\n\n    def on_train_begin(self, logs={}):\n        self.acc = []\n        self.loss = []\n        self.val_loss = []\n        self.val_acc = []\n\n    def on_train_end(self, logs={}):\n        return\n\n    def on_epoch_begin(self, epoch, logs={}):\n        return\n\n    def on_epoch_end(self, epoch, logs={}):\n        self.acc.append(logs[\'acc\'])\n        self.loss.append(logs[\'loss\'])\n        self.val_acc.append(logs[\'val_acc\'])\n        self.val_loss.append(logs[\'val_loss\'])\n        \n        pl.hold(True)\n        plt.rcParams["figure.figsize"] = (8,5)\n        pl.plot(self.acc, \'r\')\n        pl.plot(self.loss, \'b\')\n        pl.plot(self.val_acc, \'g\')\n        pl.plot(self.val_loss, \'k\')\n        pl.legend([\'Train acc\',\'Train loss\',\'Valid acc\', \'Valid loss\'], loc=2)\n        display.clear_output(wait=Tr

In [6]:
WEIGHT_DECAY = 0.0005

loss_metric = "categorical_crossentropy"
model.compile(loss=loss_metric, metrics=["accuracy"], 
              optimizer=optimizers.Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=WEIGHT_DECAY, amsgrad=False))

routine_dir = model_dir + "routine-{epoch:02d}-{acc:.2f}.hdf5"
save_models = ModelCheckpoint(routine_dir, monitor='acc', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=10)
best_dir = model_dir + "best-{epoch:02d}-{acc:.2f}.hdf5"
save_best = ModelCheckpoint(best_dir, monitor='acc', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
#histories = Histories()
res = model.fit_generator(train_generator, steps_per_epoch = math.ceil(5075/BATCH_SIZE), epochs=190, verbose=2, callbacks=[save_models, save_best])


Epoch 1/190
 - 11s - loss: 3.0004 - acc: 0.1573
Epoch 2/190
 - 10s - loss: 1.1771 - acc: 0.5520
Epoch 3/190
 - 11s - loss: 0.6384 - acc: 0.7593
Epoch 4/190
 - 10s - loss: 0.4108 - acc: 0.8625
Epoch 5/190
 - 11s - loss: 0.3637 - acc: 0.8832
Epoch 6/190
 - 11s - loss: 0.3606 - acc: 0.8861
Epoch 7/190
 - 11s - loss: 0.2085 - acc: 0.9452
Epoch 8/190
 - 11s - loss: 0.2255 - acc: 0.9311
Epoch 9/190
 - 11s - loss: 0.1732 - acc: 0.9516
Epoch 10/190
 - 10s - loss: 0.1486 - acc: 0.9626
Epoch 11/190
 - 11s - loss: 0.1603 - acc: 0.9562
Epoch 12/190
 - 11s - loss: 0.1552 - acc: 0.9581
Epoch 13/190
 - 11s - loss: 0.1469 - acc: 0.9571
Epoch 14/190
 - 11s - loss: 0.1618 - acc: 0.9539
Epoch 15/190
 - 11s - loss: 0.1400 - acc: 0.9573
Epoch 16/190
 - 10s - loss: 0.1175 - acc: 0.9669
Epoch 17/190
 - 9s - loss: 0.0898 - acc: 0.9768
Epoch 18/190
 - 9s - loss: 0.0814 - acc: 0.9774
Epoch 19/190
 - 9s - loss: 0.0913 - acc: 0.9730
Epoch 20/190
 - 9s - loss: 0.0903 - acc: 0.9717
Epoch 21/190
 - 9s - loss: 0.0887

Epoch 165/190
 - 8s - loss: 0.0056 - acc: 0.9980
Epoch 166/190
 - 8s - loss: 0.0030 - acc: 0.9988
Epoch 167/190
 - 9s - loss: 0.0030 - acc: 0.9992
Epoch 168/190
 - 8s - loss: 0.0064 - acc: 0.9979
Epoch 169/190
 - 8s - loss: 0.0229 - acc: 0.9947
Epoch 170/190
 - 9s - loss: 0.0149 - acc: 0.9953
Epoch 171/190
 - 9s - loss: 0.0028 - acc: 0.9990
Epoch 172/190
 - 9s - loss: 0.0025 - acc: 0.9991
Epoch 173/190
 - 9s - loss: 0.0064 - acc: 0.9986
Epoch 174/190
 - 9s - loss: 0.0034 - acc: 0.9987
Epoch 175/190
 - 9s - loss: 0.0012 - acc: 0.9996
Epoch 176/190
 - 9s - loss: 0.0019 - acc: 0.9988
Epoch 177/190
 - 9s - loss: 0.0372 - acc: 0.9928
Epoch 178/190
 - 8s - loss: 0.0195 - acc: 0.9959
Epoch 179/190
 - 9s - loss: 0.0056 - acc: 0.9984
Epoch 180/190
 - 9s - loss: 0.0025 - acc: 0.9990
Epoch 181/190
 - 9s - loss: 0.0016 - acc: 0.9994
Epoch 182/190
 - 9s - loss: 0.0014 - acc: 0.9994
Epoch 183/190
 - 9s - loss: 0.0024 - acc: 0.9994
Epoch 184/190
 - 8s - loss: 0.0046 - acc: 0.9990
Epoch 185/190
 - 9s 

In [ ]:
scores = model.evaluate(x_train, y_train, verbose=0)
print("Train Accuracy: %.2f%%" % (scores[1]*100))
scores = model.evaluate(x_test, y_test, verbose=0)
print("Test Accuracy: %.2f%%" % (scores[1]*100))

In [7]:
#from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1049061363234677146
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 79508275
locality {
  bus_id: 1
}
incarnation: 10124132242725038463
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:65:00.0, compute capability: 6.1"
]
